In [1]:
import requests

OLLAMA_URL = "http://localhost:11434/api/generate"

def agent1(brd_text: str) -> str:
    prd_instruction = """You are a Product Manager, but keep your output minimal and developer-focused. 
    Convert the following Business Requirement Document (BRD) into a concise Frontend Product Requirement Document (PRD)
    Rules:
    - Be short and direct. 
    - No long explanations or repeated sentences. 
    - Only include details needed for developers to build the frontend. 
    - Focus on structure, not prose. 
    
    Your PRD must have these exact sections:
    1. Objectives (1-3 bullet points max)
    2. UI Screens & Components (list of screens and key UI components only)
    3. State Management (list the slices/states required)
    4. Data Persistence (only if applicable)
    5. User Roles (if any, otherwise say 'None')
    6. Acceptance Criteria (just the checklist of frontend behaviors)
    
    Output should be structured and minimal.
    BRD:
    """

    payload = {'model':'llama3',
               'prompt': prd_instruction + brd_text,
               'stream':False}
    
    resp = requests.post(OLLAMA_URL, json=payload).json()
    return resp.get('response','').strip()

def agent2(prd_text:str) -> str:
    arch_instruction = """You are a Solutions Architect. Convert the following Product Requirement Document (PRD)
    into a developer-focused **Frontend Architecture Specification** that explicitly maps features to file paths.
    
    Rules:
    - The frontend must always be implemented in **React**, regardless of what the PRD says.
    - Styling must use **Tailwind CSS**.
    - Build tool must be **Vite**.
    - State management should default to **Redux Toolkit** (or Context API if very simple).
    - Client-side storage should be **localStorage/sessionStorage** if data persistence is needed.
    - All screens and components must be explicitly mapped to file paths (e.g., src/screens/Home.jsx, src/components/Navbar.jsx).
    - Mandatory base files must always be listed under "Mandatory Files".
    
    Your architecture document must contain ONLY the following sections:
    1. Tech Stack (React, Tailwind, Vite, state management, other required libs)
    2. Mandatory Files (always present: src/App.jsx, src/main.jsx, src/index.css, tailwind.config.js, vite.config.js, package.json)
    3. Screens (each screen listed in PRD → mapped to src/screens/*.jsx)
    4. Components (each reusable component → mapped to src/components/*.jsx)
    5. State Management (Redux store and slices if needed, with file paths)
    6. Data Storage (localStorage/sessionStorage if applicable, otherwise None)
    7. UI Features (responsive design, animations, forms, role-based access if mentioned)
    
    Output must be a clean markdown document with file mappings.
    """

    payload = {'model': 'qwen2.5-coder:7b',
               'prompt': arch_instruction + prd_text,
               'stream': False,
               'options': {"num_ctx": 4096}}
    
    resp = requests.post(OLLAMA_URL, json = payload).json()
    return resp.get('response','').strip()


def agent3(arch_text: str) -> str:
    code_instruction = """You are a Frontend Developer. Output a single VALID JSON object.
    Top-level key MUST be "frontend". Each key is a file path, each value is the full file content as a string.
    Absolutely NO markdown, NO code fences, NO comments, NO prose. Just JSON.

    Use ONLY: React, Redux Toolkit (if state management needed), Tailwind CSS, Vite, PapaParse, uuid.
    Always use React + Tailwind for UI. No other frontend frameworks allowed.
    Do NOT include axios, react-toastify, or any other libraries.

    Rules:
    - Mandatory files (must always be present): 
      src/App.jsx, src/main.jsx, src/index.css, tailwind.config.js, vite.config.js, package.json
    - Also mandatory if PRD/Architecture specifies them (NOT optional): 
      src/store/store.js (if state management is mentioned),
      src/screens/* (for each screen listed in architecture),
      src/components/* (for each component listed in architecture)
    - The structure of components, screens, and Redux slices MUST be based on the entities, features, and navigation described in the PRD/arch_text.
    - If state persistence is needed, use localStorage (load on init, save on changes).
    - For navigation:
      - If PRD says "single-page scroll", use section IDs + anchor links in Navbar (no react-router-dom).
      - If PRD says "multi-page routing", then use react-router-dom v6.
    - All components must be implemented as React functional components with JSX.
    - Every file value MUST contain actual valid source code (imports, functions, JSX, etc.). 
      Do NOT output summaries, placeholders, or meta descriptions.
    - package.json MUST only include the allowed deps above.
    - Use double quotes in JSON. Escape all newlines inside strings as \\n. 
    - No trailing commas anywhere in JSON or code strings.

    The output must strictly follow the architecture specifications provided in arch_text.
    """

    payload ={'model': 'qwen2.5-coder:7b',
              'stream' : False,
              'prompt': code_instruction + arch_text,
              'options': {"num_ctx": 8192}}
    
    resp = requests.post(OLLAMA_URL,json = payload).json()
    return resp.get('response','').strip()


brd_text = """We want to build a personal portfolio website for showcasing an individual's projects and skills. The website should have the following sections: Home, About, Projects, Skills, and Contact. Users should be able to click through navigation links at the top of the page to move between sections smoothly (single-page scrolling). 
    The Projects section should display project cards with images, titles, short descriptions, and buttons to view more details or visit external links (e.g., GitHub, Live Demo). 
    The Contact section should include a simple form with fields for name, email, and message (no backend processing required, just UI). 
    The design should be modern, responsive (desktop, tablet, mobile), and lightweight, with a visually appealing color scheme. 
    Animations and transitions should be used for smooth user experience. 
    There is no need for backend or database; this is a frontend-only project."""


prd_output = agent1(brd_text)
print("\n=== PRD ===\n", prd_output)

arch_output = agent2(prd_output)
print("\n=== Arch ===\n", arch_output)

code_output = agent3(arch_output)
print("\n=== Generated code ===\n", code_output)


=== PRD ===
 Here is the converted PRD:

**Objectives**

• Build a modern, responsive personal portfolio website
• Showcase projects and skills with smooth navigation between sections
• Create a visually appealing design with animations and transitions

**UI Screens & Components**

• Home
• About
• Projects (project cards with images, titles, descriptions, and buttons)
• Skills
• Contact (simple form with name, email, and message fields)

**State Management**

• Navigation: active section state management
• Project card: expanded/collapsed state management

**Data Persistence**
None (frontend-only project)

**User Roles**
None

**Acceptance Criteria**

• Single-page scrolling between sections
• Smooth navigation using top-level links
• Projects section displays project cards with images, titles, descriptions, and buttons
• Contact section includes a simple form with name, email, and message fields
• Responsive design for desktop, tablet, and mobile devices
• Animations and transitions

In [ ]:
brd_text = """...your BRD input..."""

prd = agent1_generate_prd(brd_text)
arch = agent2_generate_architecture(brd_text, prd)
code = agent3_generate_code(brd_text, prd, arch)

print("=== PRD ===\n", prd)
print("\n=== Architecture ===\n", arch)
print("\n=== Code ===\n", code)
